#### LangChain Essentials Course

# LangChain Agent Executor

In this chapter, we will continue from the [introduction to agents](https://aurelio.ai/learn/langchain-agents-intro) and dive deeper into agents. Learning how to build our custom agent execution loop for v0.3 of LangChain.

In [1]:
!pip install -qU \
  langchain-core==0.3.33 \
  langchain-openai==0.3.3 \
  langchain-community==0.3.16 \
  langsmith==0.3.4 \
  langchain-ollama

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 731.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 333.3/333.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00


In [2]:
!apt update && apt install -y pciutils lshw

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,681 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,237 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8

---

> ⚠️ We will be using OpenAI for this example allowing us to run everything via API. If you would like to use Ollama instead, check out the [Ollama LangChain Course](https://github.com/aurelio-labs/langchain-course/tree/main/notebooks/ollama).

---

In [3]:

from google.colab import userdata
Lang_api =userdata.get('Langsmith_api')

---

> ⚠️ If using LangSmith, add your API key below:

In [4]:
import os
from getpass import getpass

# must enter API key
os.environ["LANGCHAIN_API_KEY"] =Lang_api

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-agent-executor-openai"

---

In [5]:
!curl -fsSL https://ollama.ai/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [6]:

!ollama serve > /dev/null 2>&1 &

In [ ]:
!ollama pull mistral

In [8]:
import os
from langchain_ollama.chat_models import ChatOllama

model_name = "mistral"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)
creative_llm = ChatOllama(temperature=0.9, model=model_name)

In [9]:
!ollama list

NAME              ID              SIZE      MODIFIED       
mistral:latest    f974a74358d6    4.1 GB    11 seconds ago    


In [10]:

llm.invoke("Tell me about virat kohli")

AIMessage(content=" Virat Kohli is a renowned Indian cricketer who was born on November 5, 1988. He is widely regarded as one of the greatest batsmen in the history of cricket. Kohli serves as the captain of the Indian national team across formats (Test, One Day International (ODI), and Twenty20 International (T20I)).\n\nKohli made his ODI debut against Bangladesh on August 18, 2007, and his T20I debut against Scotland on June 3, 2007. He made his Test debut against West Indies in Bangalore on November 20, 2011.\n\nKohli has been the recipient of numerous awards, including the Rajiv Gandhi Khel Ratna award in 2018, India's highest honor for sportspeople, and the Arjuna Award in 2013. He was also named ICC Cricketer of the Year and ODI Cricketer of the Year in 2012.\n\nKohli is known for his aggressive playing style and exceptional consistency. As of March 2023, he ranks second on the list of all-time run scorers in both ODIs and T20Is. In Tests, he has scored more than 7,500 runs with 

## What is the Agent Executor?

When we talk about agents, a significant part of an "agent" is simple code logic,
iteratively rerunning LLM calls and processing their output. The exact logic varies
significantly, but one well-known example is the **ReAct** agent.

![ReAct process](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-00.png&w=640&q=75)

**Re**ason + **Act**ion (ReAct) agents use iterative _reasoning_ and _action_ steps to
incorporate chain-of-thought and tool-use into their execution. During the _reasoning_
step, the LLM generates the steps to take to answer the query. Next, the LLM generates
the _action_ input, which our code logic parses into a tool call.

![Agentic graph of ReAct](https://www.aurelio.ai/_next/image?url=%2Fimages%2Fposts%2Fai-agents%2Fai-agents-01.png&w=640&q=75)

Following our action step, we get an observation from the tool call. Then, we feed the
observation back into the agent executor logic for a final answer or further reasoning
and action steps.

The agent and agent executor we will be building will follow this pattern.

## Tools

As with the previous chapter, we will define a few tools to use within our agent.

In [11]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

# Define the multiply tool
@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

# Define the exponentiate tool
@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [12]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7db3b4434ae0>)

We can see the tool name, description, and arg schema:

In [13]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [14]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

The `args_schema` is a pydantic model that is transformed into the JSON schema above and passed into our LLM, it is this that defines _how_ the tool is used for the LLM. We can see this with other tools:

In [15]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [16]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [17]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent writes its notes as it works through multiple internal thought and tool-use steps to produce a final output for the user. This scratchpad is a list of messages with alternating roles of `ai` (for the tool call) and `tool` (for the tool execution output). Both message types require a `tool_call_id` field which is used to link the respective AI and tool messages - this can be required when we many tool calls happening in parallel.

In [18]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

Next, we must define our LLM, we will use the `gpt-4o-mini` model with a `temperature` of `0.0`.

In [19]:
# import os
# from getpass import getpass
# from langchain_openai import ChatOpenAI

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY") \
#     or getpass("Enter your OpenAI API key: ")

# llm = ChatOpenAI(
#     model_name="gpt-4o-mini",
#     temperature=0.0,
# )

To add tools to our LLM, we will use the `bind_tools` method within the LCEL constructor, which will take our tools and add them to the LLM. We'll also include the `tool_choice="any"` argument to `bind_tools`, which tells the LLM that it _MUST_ use a tool, ie it cannot provide a final answer directly (in therefore not using a tool):

In [20]:
from langchain_core.runnables.base import RunnableSerializable

tools = [add, subtract, multiply, exponentiate]

# define the agent runnable
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="required")
)

We invoke the agent with the `invoke` method, passing in the input and chat history.

In [21]:
tool_call = agent.invoke({"input": "What is 1345 * 9283?", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-23T12:17:48.08649988Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2472305222, 'load_duration': 11305030, 'prompt_eval_count': 367, 'prompt_eval_duration': 133270198, 'eval_count': 91, 'eval_duration': 2325230939, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-f521614a-1125-44e1-8296-892d251a8fc8-0', tool_calls=[{'name': 'multiply', 'args': {'x': 1345, 'y': 9283}, 'id': 'bd288ce1-dd0d-46ec-8e54-71b7efdb5510', 'type': 'tool_call'}], usage_metadata={'input_tokens': 367, 'output_tokens': 91, 'total_tokens': 458})

Because we set `tool_choice="any"` to force the tool output, the usual `content` field will be empty as that field is used for natural language output, ie the _final answer_ of the LLM. To find our tool output, we need to look at the `tool_calls` field:

In [22]:
tool_call.tool_calls

[{'name': 'multiply',
  'args': {'x': 1345, 'y': 9283},
  'id': 'bd288ce1-dd0d-46ec-8e54-71b7efdb5510',
  'type': 'tool_call'}]

From here, we have the tool `name` that our LLM wants to use and the `args` that it
wants to pass to that tool. We can see that the tool `add` is being used with the
arguments `x=10` and `y=10`. The `agent.invoke` method has _not_ executed the tool
function; we need to write that part of the agent code ourselves.

Executing the tool code requires two steps:

1. Map the tool `name` to the tool function.

2. Execute the tool function with the generated `args`.

In [23]:
# create tool name to function mapping
name2tool = {tool.name: tool.func for tool in tools}

Now execute to get our answer:

In [24]:
tool_exec_content = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)
tool_exec_content

12485635

That is our answer and tool execution logic. We feed this back into our LLM via the
`agent_scratchpad` placeholder.

In [25]:
from langchain_core.messages import ToolMessage

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_exec_content}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 1345 * 9283?",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='1345 * 9283 = 12485635.', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-23T12:18:30.075911143Z', 'done': True, 'done_reason': 'stop', 'total_duration': 626600534, 'load_duration': 7833956, 'prompt_eval_count': 71, 'prompt_eval_duration': 34463569, 'eval_count': 22, 'eval_duration': 582698994, 'message': Message(role='assistant', content='1345 * 9283 = 12485635.', images=None, tool_calls=None)}, id='run-302883d8-7822-42c1-b443-1e2f930dffef-0', usage_metadata={'input_tokens': 71, 'output_tokens': 22, 'total_tokens': 93})

Despite having the answer in our `agent_scratchpad`, the LLM still tries to use the tool
_again_. This behaviour happens because we bonded the tools to the LLM with
`tool_choice="any"`. When we set `tool_choice` to `"any"` or `"required"`, we tell the
LLM that it _MUST_ use a tool, i.e., it cannot provide a final answer.

There's two options to fix this:

1. Set `tool_choice="auto"` to tell the LLM that it can choose to use a tool or provide
a final answer.

2. Create a `final_answer` tool - we'll explain this shortly.

First, let's try option **1**:

In [36]:
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="auto")
)

We'll start from the start again, so `agent_scratchpad` is empty:

In [37]:
tool_call = agent.invoke({"input": "What is 1345 * 9283?", "chat_history": []})
tool_call

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-23T12:20:02.675521866Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2577203024, 'load_duration': 13261674, 'prompt_eval_count': 367, 'prompt_eval_duration': 134038149, 'eval_count': 91, 'eval_duration': 2427466526, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6d1becf1-b0c1-4e7a-8fa7-1bf6cb24f098-0', tool_calls=[{'name': 'multiply', 'args': {'x': 1345, 'y': 9283}, 'id': '4e655e64-41e0-439f-bff0-d1e42e704561', 'type': 'tool_call'}], usage_metadata={'input_tokens': 367, 'output_tokens': 91, 'total_tokens': 458})

Now we execute the tool and pass it's output into the `agent_scratchpad` placeholder:

In [38]:
tool_output = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_output}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input":  "What is 1345 * 9283?",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='1345 * 9283 = 12485635.', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-23T12:20:08.644589538Z', 'done': True, 'done_reason': 'stop', 'total_duration': 631469350, 'load_duration': 8010268, 'prompt_eval_count': 71, 'prompt_eval_duration': 34029068, 'eval_count': 22, 'eval_duration': 587809407, 'message': Message(role='assistant', content='1345 * 9283 = 12485635.', images=None, tool_calls=None)}, id='run-0fc74841-cedb-4650-a720-3216bbee21df-0', usage_metadata={'input_tokens': 71, 'output_tokens': 22, 'total_tokens': 93})

We now have the final answer in the `content` field! This method is perfectly
functional; however, we recommend option **2** as it provides more control over the
agent's output.

There are several reasons that option **2** can provide more control, those are:

* It removes the possibility of an agent using the direct `content` field when it is not
appropriate; for example, some LLMs (particularly smaller ones) may try to use the
`content` field when using a tool.

* We can enforce a specific structured output in our answers. Structured outputs are
handy when we require particular fields for downstream code or multi-part answers. For
example, a RAG agent may return a natural language answer and a list of sources used to
generate that answer.

To implement option **2**, we must create a `final_answer` tool. We will add a
`tools_used` field to give our output some structure—in a real-world use case, we
probably wouldn't want to generate this field, but it's useful for our example here.

In [39]:
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

Our `final_answer` tool _doesn't_ necessarily need to do anything; in this example,
we're using it purely to structure our final response. We can now add this tool to our
agent:

In [40]:
tools = [final_answer, add, subtract, multiply, exponentiate]

# we need to update our name2tool mapping too
name2tool = {tool.name: tool.func for tool in tools}

agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | prompt
    | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
)

Now we invoke:

In [41]:
tool_call = agent.invoke({"input": "What is 10 + 10", "chat_history": []})
tool_call.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'ce665e17-9cef-4f36-94af-ba877275d5a4',
  'type': 'tool_call'}]

We execute the tool and provide it's output to the agent again:

In [45]:
tool_out = name2tool[tool_call.tool_calls[0]["name"]](
    **tool_call.tool_calls[0]["args"]
)

tool_exec = ToolMessage(
    content=f"The {tool_call.tool_calls[0]['name']} tool returned {tool_out}",
    tool_call_id=tool_call.tool_calls[0]["id"]
)

out = agent.invoke({
    "input": "What is 10 + 10",
    "chat_history": [],
    "agent_scratchpad": [tool_call, tool_exec]
})
out

AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'mistral', 'created_at': '2025-03-23T12:21:10.109516127Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1159642566, 'load_duration': 10621684, 'prompt_eval_count': 54, 'prompt_eval_duration': 5813380, 'eval_count': 48, 'eval_duration': 1141649860, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-3cccc868-cf6b-4db9-bbda-48c90bd3fa00-0', tool_calls=[{'name': 'add', 'args': {'x': 10, 'y': 10}, 'id': 'b29f5eb5-51a7-4d5c-a351-2c4ab1d1350f', 'type': 'tool_call'}], usage_metadata={'input_tokens': 54, 'output_tokens': 48, 'total_tokens': 102})

We see that `content` remains empty because we force tool use. But we now have the
`final_answer` tool, which the agent executor passes via the `tool_calls` field:

In [46]:
out.tool_calls

[{'name': 'add',
  'args': {'x': 10, 'y': 10},
  'id': 'b29f5eb5-51a7-4d5c-a351-2c4ab1d1350f',
  'type': 'tool_call'}]

Because we see the `final_answer` tool here, we don't pass this back into our agent, and
instead, this tells us to stop execution and pass the `args` output onto our downstream
process or user directly:

In [47]:
out.tool_calls[0]["args"]

{'x': 10, 'y': 10}

### Building a Custom Agent Execution Loop

We've worked through each step of our agent code, but it doesn't run without us running
every step. We must write a class to handle all the logic we just worked through.

In [61]:
tool_out = None

In [79]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage


class CustomAgentExecutor:
    chat_history: list[BaseMessage]

    def __init__(self, max_iterations: int = 3):
        self.chat_history = []
        self.max_iterations = max_iterations
        self.agent: RunnableSerializable = (
            {
                "input": lambda x: x["input"],
                "chat_history": lambda x: x["chat_history"],
                "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
            }
            | prompt
            | llm.bind_tools(tools, tool_choice="any")  # we're forcing tool use again
        )

    def invoke(self, input: str) -> dict:
        # invoke the agent but we do this iteratively in a loop until
        # reaching a final answer
        count = 0
        agent_scratchpad = []
        while count < self.max_iterations:
            # invoke a step for the agent to generate a tool call
            tool_call = self.agent.invoke({
                "input": input,
                "chat_history": self.chat_history,
                "agent_scratchpad": agent_scratchpad
            })
            # add initial tool call to scratchpad
            agent_scratchpad.append(tool_call)
            # otherwise we execute the tool and add it's output to the agent scratchpad
            if len(tool_call.tool_calls)==0:
               break
            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = name2tool[tool_name](**tool_args)
            # add the tool output to the agent scratchpad
            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)
            # add a print so we can see intermediate steps
            print(f"{count}: {tool_name}({tool_args})")
            count += 1
            # if the tool call is the final answer tool, we stop
            if tool_name == "final_answer":
                break
        # add the final output to the chat history
        # print(tool_out)
        # final_answer = tool_out["answer"]
        self.chat_history.extend([
            HumanMessage(content=input),
            AIMessage(content=tool_call.content)
        ])
        # return the final answer in dict form
        return json.dumps(tool_out)

Now initialize the agent executor:

In [80]:
agent_executor = CustomAgentExecutor()

And test the `invoke` method:

In [81]:
agent_executor.invoke(input="What is 10 + 10")

0: add({'x': 10, 'y': 10})


'20'

We then get our answer and the tools that were used — all through our custom agent
executor.